In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
import time

# Observação: o driver será criado/fechado no bloco que itera as páginas,
# para evitarmos criar/fechar o Chrome a cada página (isso reduz falhas intermitentes).
# Aqui deixamos apenas as importações e utilitários.

In [3]:
def extract_text_from_page(url, driver, max_wait=10, check_interval=0.05):
    """
    Extrai 'titulo' e 'conteudo' com polling inteligente baseado em marcador.
    
    Estratégia:
    - Aguarda elemento aparecer
    - Polling frequente (50ms) verificando se conteúdo começa com 'COD_TIPO:'
    - Retorna IMEDIATAMENTE assim que marcador é detectado (sem wait desnecessário)
    - Max wait total de 10s como fallback
    """
    start_time = time.time()
    
    try:
        driver.get(url)
        
        # Aguarda elemento aparecer no DOM
        wait = WebDriverWait(driver, max_wait)
        wait.until(EC.presence_of_element_located((By.ID, "TextoDigitadoTxt")))
        
        # ============ POLLING INTELIGENTE COM MARCADOR ============
        conteudo = None
        content_found = False
        
        while time.time() - start_time < max_wait:
            try:
                conteudo = driver.execute_script("""
                    var el = document.getElementById('TextoDigitadoTxt');
                    if (!el) return null;
                    var text = el.value || el.innerText || el.textContent || '';
                    return text.trim() || null;
                """)
                
                # Verifica se tem o marcador (conteúdo pronto)
                if conteudo and 'COD_TIPO:' in conteudo:
                    content_found = True
                    break
                elif conteudo:
                    # Tem algo mas sem marcador - aguarda mais um pouco
                    time.sleep(check_interval)
                else:
                    # Vazio ainda - aguarda
                    time.sleep(check_interval)
                    
            except Exception:
                time.sleep(check_interval)
        
        # ============ EXTRAÇÃO DE TÍTULO ============
        texto_pasta = None
        try:
            pasta_el = driver.find_element(By.ID, "PastaTxt")
            texto_pasta = pasta_el.get_attribute("innerText") or pasta_el.text or ''
            texto_pasta = texto_pasta.strip() if texto_pasta else None
        except Exception:
            pass

        # ============ RETORNO ============
        return {
            "titulo": texto_pasta,
            "conteudo": conteudo.strip() if conteudo else None
        }

    except TimeoutException:
        return {"titulo": None, "conteudo": None}
    except Exception:
        return {"titulo": None, "conteudo": None}

In [4]:
import pandas as pd
df_dados = pd.DataFrame(columns=["titulo", "conteudo"])

In [5]:
# Cria o driver uma vez e o reutiliza para todas as páginas

# estimativa de tempo = 9126 * 3.7 seg
# 33772.2 segundos = 562.87 minutos = 9.38 horas = 9h e 22m
print("Iniciando driver Chrome...")
#print("Estimativa de tempo: 9h e 22m")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
try:
    print("=" * 80)
    print("INICIANDO SCRAPING - Polling Inteligente (COD_TIPO:)")
    print("=" * 80)
    times = []
    for i in range(8271, 9127):
        url = f"https://www.docvirt.com/docreader.net/DocReader.aspx?bib=coportinari&id=4645702686696&pagfis={i}"
        print(f"[{i}/9126]", flush=True)

        # Limpa storage
        try:
            driver.execute_script("window.localStorage.clear(); window.sessionStorage.clear();")
        except Exception:
            pass
        
        # Extrai com polling inteligente
        t0 = time.time()
        linha_df = extract_text_from_page(url, driver, max_wait=10, check_interval=0.05)
        elapsed = time.time() - t0
        times.append(elapsed)
        
        df_dados = pd.concat([df_dados, pd.DataFrame([linha_df])], ignore_index=True)
        
        # Status com tempo
        status = "✓" if (linha_df['titulo'] or linha_df['conteudo']) else "✗"
        
        # Delay mínimo
        time.sleep(0.1)

    print("\n", "=" * 80)
    print(f"✓ Concluído: {len(df_dados)} páginas em {sum(times):.2f}s (média: {sum(times)/len(times):.2f}s/página)")
    print("=" * 80)
    
    # Salva ao fim
    df_dados.to_csv("dados_extraidos2.csv", index=False)
finally:
    driver.quit()

Iniciando driver Chrome...
Estimativa de tempo: 9h e 22m
INICIANDO SCRAPING - Polling Inteligente (COD_TIPO:)
[8271/9126]
[8272/9126]
[8273/9126]
[8274/9126]
[8275/9126]
[8276/9126]
[8277/9126]
[8278/9126]
[8279/9126]
[8280/9126]
[8281/9126]
[8282/9126]
[8283/9126]
[8284/9126]
[8285/9126]
[8286/9126]
[8287/9126]
[8288/9126]
[8289/9126]
[8290/9126]
[8291/9126]
[8292/9126]
[8293/9126]
[8294/9126]
[8295/9126]
[8296/9126]
[8297/9126]
[8298/9126]
[8299/9126]
[8300/9126]
[8301/9126]
[8302/9126]
[8303/9126]
[8304/9126]
[8305/9126]
[8306/9126]
[8307/9126]
[8308/9126]
[8309/9126]
[8310/9126]
[8311/9126]
[8312/9126]
[8313/9126]
[8314/9126]
[8315/9126]
[8316/9126]
[8317/9126]
[8318/9126]
[8319/9126]
[8320/9126]
[8321/9126]
[8322/9126]
[8323/9126]
[8324/9126]
[8325/9126]
[8326/9126]
[8327/9126]
[8328/9126]
[8329/9126]
[8330/9126]
[8331/9126]
[8332/9126]
[8333/9126]
[8334/9126]
[8335/9126]
[8336/9126]
[8337/9126]
[8338/9126]
[8339/9126]
[8340/9126]
[8341/9126]
[8342/9126]
[8343/9126]
[8344/9126]
[8